In [ ]:
from functools import reduce

In [ ]:
import pandas as pd
import numpy as np
#import openpyxl
import requests
import json
import pandas as pd
from os import path
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.stats

The file contains the DAO v0.3 evaluation, including the HOPR forum plugins, and the votes on snapshot 
CSV file's numbers always refer to the forum plugins, from where they were extracted

The abbreviatins follow the logic of the allocation board on the drive: https://docs.google.com/spreadsheets/d/1netmY3CkYo06PxdDMv4jqifty24Y6gBnHQko0ZyVmzE/edit#gid=0 

Winning proposal 	P_W_P
Winning siniture 	P_W_S
Vote stage not winning**	P_V_P
Vote stage signitures 	P_V_S
Referendum stage not voting	P_R_P
Referendum stage not voting signitures	P_R_S
Ten posts with most support	D_P_L
Ten users with most total likes across discussion	D_U_L
Users with three posts with 3+ likes each	U_T_T
At least one of contribution	M_O_C
Highest efficiency ratio	M_E_R
Consistent contribution	M_C
Moderator power	M_M
Best technical contribution	T_C


TO DO LIST:

- changing the FORUM CSV files to new ones - checking the final amount 
- modifying the m_m user_ids, and the calculation of the 5% share, based on the number of rewarded participants
- remowing duplications at the of the code 

In [ ]:
#proposals and their status
#manually created file
df_proposals = pd.read_csv('dao_v03_1_proposals.csv')
df_proposals

In [ ]:
#creating a column for proposals being considered as valid
df_proposals["ref"] = 1
refer = df_proposals[["post_id", "ref"]]

In [ ]:
#signitures given to proposals
df_signitures = pd.read_csv('dao_v03_5_distributions.csv')
df_signitures

In [ ]:
#dropping signatures which are not considered
df_signitures = pd.merge(df_signitures, refer, how="outer", on=["post_id"])

In [ ]:
df_signitures = df_signitures[df_signitures['ref'].notna()]
df_signitures.head(2)

In [ ]:
#mergind the proposal and the signiture files
df_merge_pr_si = pd.merge(df_signitures, df_proposals, how="outer", on=["topic_id"])

In [ ]:
# checking the max number of signitures - if someone liked more than 8, must be out
df_merge_pr_si["user_id_x"].value_counts().reset_index()

### WINNING PROPOSAL P-W-P

In [ ]:
#winning proposal P_W_P
p_w_p = df_proposals[df_proposals['winning'] == 1].reset_index(drop=True)
p_w_p = p_w_p[['user_id','winning']]
p_w_p

In [ ]:
#post_id of winning proposal
winner_post_id = [11019]

### WINNING SIGNITORIES P-W-S

In [ ]:
#celectingt the signitories which sign the winning proposal
p_w_s = df_signitures.loc[df_signitures['post_id'].isin(winner_post_id)].reset_index(drop=True)

In [ ]:
#dropping the proposer from the list of signatories
p_w_s = p_w_s[p_w_s['user_id'] != 544]

In [ ]:
p_w_s = p_w_s[['user_id','ref']].rename(columns={'ref': 'winning_signitures'}).reset_index(drop=True)

In [ ]:
#summing up by user_id
p_w_s = p_w_s.groupby(by="user_id").sum().reset_index(drop=False)
p_w_s

### VOTE STAGE PROPOSALS - P_V_P

In [ ]:
#creating a dataframe from the proposals
df_vote_stage_pr = df_proposals[df_proposals['vote_stage'] == 1].reset_index(drop=True)
df_vote_stage_pr.head(5)

In [ ]:
#vote stage proposers
p_v_p = df_vote_stage_pr[['user_id','vote_stage']]

In [ ]:
#summing up by user_id
p_v_p = p_v_p.groupby(by="user_id").sum().reset_index(drop=False)
p_v_p

### VOTE STAGE SIGNITURES - P_V_S

In [ ]:
#creating a dataframe from the signitures
df_vote_stage_si = df_signitures.loc[df_signitures['post_id'].isin(df_vote_stage_pr['post_id'])].reset_index(drop=True)

In [ ]:
df_vote_stage_si.head(5)

In [ ]:
# adding a variable, which is a combination of post id and user id, to remove proposers form the signitories
df_vote_stage_si = df_vote_stage_si.assign(combi2 = (df_vote_stage_si['post_id'].apply(lambda x: str(x))) + (df_vote_stage_si['user_id'].apply(lambda x: str(x))))

In [ ]:
#list of proposers
p_v_s_proposers = df_vote_stage_pr[['user_id', 'post_id']]
p_v_s_proposers.head(5)

In [ ]:
#adding the same variable to the list of proposers
p_v_s_proposers = p_v_s_proposers.assign(combi2 = (p_v_s_proposers['post_id'].apply(lambda x: str(x))) + (p_v_s_proposers['user_id'].apply(lambda x: str(x))))

In [ ]:
p_v_s_proposers.head(5)

In [ ]:
#removing the proposers
vote_signitures_without_proposers = df_vote_stage_si.loc[~df_vote_stage_si['combi2'].isin(p_v_s_proposers['combi2'])].reset_index(drop=True)

In [ ]:
vote_signitures_without_proposers.head(5)

In [ ]:
# signitures given for the not winning vote stage proposals
p_v_s = vote_signitures_without_proposers[['user_id', 'ref']]
p_v_s = p_v_s.rename(columns={'ref': 'vote_stage_signitures'}).reset_index(drop=True)
p_v_s

In [ ]:
#summing up the signitures
p_v_s = p_v_s.groupby(by="user_id").sum().reset_index(drop=False)
p_v_s

### REFERENDUM STAGE PROPOSALS - P_R_P

In [ ]:
#proposals
df_no_vote_pr = df_proposals[df_proposals['no_vote'] == 1].reset_index(drop=True)

In [ ]:
#no_vote stage proposals
p_r_p = df_no_vote_pr[['user_id','no_vote']]

In [ ]:
#summing up the proposals by user_id
p_r_p = p_r_p.groupby(by="user_id").sum().reset_index(drop=False)
p_r_p

### REFERENDUM STAGE SIGNITURES - P_R_S

In [ ]:
df_no_vote_si = df_no_vote_pr[['user_id','post_id','no_vote']]

In [ ]:
#creating a variable to remove the proposers
df_no_vote_si = df_no_vote_si.assign(combi2 = (df_no_vote_si['post_id'].apply(lambda x: str(x))) + (df_no_vote_si['user_id'].apply(lambda x: str(x))))

In [ ]:
# proposal's post_id
df_no_vote_signitures = df_signitures.loc[df_signitures['post_id'].isin(df_no_vote_pr['post_id'])].reset_index(drop=True)

In [ ]:
#creating a variable to remove the proposers
df_no_vote_signitures = df_no_vote_signitures.assign(combi2 = (df_no_vote_signitures['post_id'].apply(lambda x: str(x))) + (df_no_vote_signitures['user_id'].apply(lambda x: str(x))))

In [ ]:
#removing proposers
no_vote_signitures_without_proposers = df_no_vote_signitures.loc[~df_no_vote_signitures['combi2'].isin(df_no_vote_si['combi2'])].reset_index(drop=True)

In [ ]:
#subfil
p_r_s = no_vote_signitures_without_proposers[['user_id', 'ref']].rename(columns={'ref': 'no_vote_signitures'}).reset_index(drop=True)

In [ ]:
#adding up the votes
p_r_s = p_r_s.groupby(by="user_id").sum().reset_index(drop=False)
p_r_s

### DISCUSSION (D_P_L, D_U_L, U_T_T)

In [ ]:
df_discussion = pd.read_csv('dao_v03_6_discussion.csv', names=['user_id', 'total_allocation', 'd_p_l', 'd_u_l', 'u_t_t'], header=0)

In [ ]:
df_discussion.head(5)

In [ ]:
df_discussion['user_id'] = df_discussion['user_id'].fillna(0)

### MISCELLANEOUS_1 - M_O_C 

In [ ]:
df_miscellaneous_1 = pd.read_csv('dao_v03_7_miscellaneous_1.csv', names=['user_id', 'm_o_c', 'count'], header=0)
df_miscellaneous_1.head(5)

### MISCELLANEOUS_2 M_E_R

In [ ]:
df_miscellaneous_2 = pd.read_csv('dao_v03_8_miscellaneous_2.csv', names=['user_id', 'coef', 'm_e_r'], header=0)
df_miscellaneous_2.head(5)

### MISCELLANEOUS_3 M_C

In [ ]:
df_miscellaneous_3 = pd.read_csv('dao_v03_9_miscellaneous_3.csv', names=['user_id', 'post_id', 'm_c', 'count'], header=0)
df_miscellaneous_3

In [ ]:
df_miscellaneous_3 = df_miscellaneous_3.groupby(by="user_id").sum().reset_index(drop=False)
df_miscellaneous_3

### MODERATOR POWER M_M

In [ ]:
#NEED TO BE MODIFIED, the user_ids were not decided yet!!!
#consider the number of participants in the share (4 participants at the moment)
m_m = pd.DataFrame({'user_id': [603, 594, 1307, 810], 'm_m': [0.05/4, 0.05/4, 0.05/4, 0.05/4]}).reset_index(drop=True)
m_m

### TECH POWER T_C

In [ ]:
t_c = pd.DataFrame({'user_id': [603, 594, 1307, 810], 't_c': [0.1/4, 0.1/4, 0.1/4, 0.1/4]}).reset_index(drop=True)

### WALET AND NFT ADDRESS

In [ ]:
wallet_address = pd.read_csv('wallet_address_10.csv', names=['id', 'username_lower', 'user_id', 'add'], header=0)
wallet_address

In [ ]:
nft_address = pd.read_csv('nft_address_11.csv', names=['id', 'username_lower', 'user_id', 'nft_add'], header=0)
nft_address

In [ ]:
#merging them
addresses = pd.merge(wallet_address, nft_address, how="outer", on=["user_id"])

In [ ]:
#creating a subfile
addresses = addresses[['user_id', 'add', 'nft_add']]
addresses

### merging categories and wallets

In [ ]:
#creating a list of dataframes from the categories
categories = [p_w_p, p_w_s, p_v_p, p_v_s, p_r_p, p_r_s, df_discussion, df_miscellaneous_1, df_miscellaneous_2, df_miscellaneous_3, m_m, t_c]

In [ ]:
#merging each element of the list, after each other, every category can add new user_id
df_merged_0 = reduce(lambda  left,right: pd.merge(left,right,on=['user_id'], how='outer'), categories)
df_merged_0

In [ ]:
#merging the dataframe with the addresses - here only the existing user_ids are consideres from the wallet list
df_merged = pd.merge(df_merged_0, addresses, how='left', on=["user_id"])
df_merged.head(5)

In [ ]:
#removing admins - user_ids which are belong to a team member

In [ ]:
#also available on the forum
admin = pd.read_csv('admin.csv', names=['user_id', 'usename'], header=0)

In [ ]:
#adding a variable with value = 1
admin.loc[:,'admin'] = 1

In [ ]:
#merging with the dataframe
df_merged = pd.merge(df_merged, admin, how="left", on=["user_id"])
df_merged

In [ ]:
#NANs changed to 0
df_merged = df_merged.fillna(0)

In [ ]:
#if the "admin" category is 1, the given used_id is removed
df_merged = df_merged[df_merged['admin'] != 1]
df_merged

### calculating coef in the proposal section - based on the allocation bard

In [ ]:
df_merged['p_w_p'] = df_merged['winning']*0.05

In [ ]:
#the allocations are shared between the users in the category, e.g. p_w_s if someone signed the winning proposal, gets the proportional share of the 5%

In [ ]:
df_merged['p_w_s'] = 0.05/sum(p_w_s["winning_signitures"])*df_merged['winning_signitures']

In [ ]:
df_merged['p_v_p'] = 0.09/sum(p_v_p["vote_stage"])*df_merged['vote_stage']

In [ ]:
df_merged['p_v_s'] = 0.06/sum(p_v_s['vote_stage_signitures'])*df_merged['vote_stage_signitures']

In [ ]:
df_merged['p_r_p'] = 0.03/sum(p_r_p['no_vote'])*df_merged['no_vote']

In [ ]:
df_merged['p_r_s'] = 0.02/sum(p_r_s["no_vote_signitures"])*df_merged['no_vote_signitures']

### calculating total share

In [ ]:
#sum of the proportional shares
df_merged['total_coef'] = df_merged['p_w_p'] + df_merged['p_w_s'] + df_merged['p_v_p'] + df_merged['p_v_s'] + df_merged['p_r_p'] + df_merged['p_r_s'] + df_merged['d_p_l'] + df_merged['d_u_l'] + df_merged['u_t_t'] + df_merged['m_o_c'] + df_merged["m_e_r"] + df_merged["m_c"] + df_merged['m_m'] + df_merged['t_c']

In [ ]:
#the proportional share from the total reward
df_merged['score'] = df_merged["total_coef"]*25000

In [ ]:
#sorting by the ammount of the reward
df_merged = (df_merged.sort_values(by='score', ascending=False)).reset_index(drop=True)
df_merged.head()

In [ ]:
#checkpoint - it should be 25000 minimal difference can be caused by the python
sum(df_merged['score'])

In [ ]:
#users without score
empty_wal = df_merged[df_merged['score'] == 0 ].index

In [ ]:
#dropping the users with no distribution
df_merged.drop(empty_wal, inplace=True)

### binning and tagging with NFT status, based on the score

In [ ]:
#creating label
b_l = ["Bronze", "Silver", "Gold", "Diamond"]
#using binning function to creat groups
df_merged["NFT_status"] = pd.qcut(df_merged["score"], 4, labels=b_l)

In [ ]:
df_merged["NFT_status"].value_counts()

### voting

In [ ]:
#url and query 
url = 'https://hub.snapshot.org/graphql?operationName=Votes'

data_h = '''
query Votes {
  votes (
    first: 1000
    skip: 0
    where: {
      proposal: "0xe02cd1e68980a1572f03de7d849c5b6c86620d6774f3a652f11069d2ad290ae9"
    }
    orderBy: "created",
    orderDirection: desc
  ) {
    id
    voter
    created
    proposal {
      id
    }
    choice
    space {
      id
    }
  }
}
'''

response_h = requests.post(url,json={'query': data_h})
#dictionary from the query
data_h = json.loads(response_h.text)

In [ ]:
#creating a datafram from the dictionary
df_h = pd.DataFrame((pd.DataFrame(data_h)).iloc[0][0])
df_h.head()

In [ ]:
from pandas.io.json import json_normalize

In [ ]:
#separating the answers from the vote - which are still dictionary objects
sample = df_h['choice']

In [ ]:
#normalizing the answers
choices = pd.json_normalize(sample)
choices.head(5)

In [ ]:
#merging back the answers to the dataframe
df_snapshot = pd.concat([df_h, choices], axis=1)

In [ ]:
#separating the voter's address
vote = pd.DataFrame(df_snapshot['voter'])

In [ ]:
#changing the characters to lower case
vote['voter'] = vote['voter'].str.lower()

In [ ]:
#changing the name of the column
vote.columns = ["nft_add"]

In [ ]:
#adding a variable as identifier after merging with the forum data
vote["voter"] = 1 
vote

In [ ]:
#changing the forum addresses to lower case
df_merged['nft_add'] = df_merged['nft_add'].str.lower()

In [ ]:
df_merged['add'] = df_merged['add'].str.lower()

### merging the addresses from the forum data and from the snapshot

In [ ]:
df_merged_vote = pd.merge(df_merged, vote, how="outer", on=["nft_add"])
df_merged_vote.head(5)

In [ ]:
#counting the addresses, which are showing up only in the voting
df_merged_vote['NFT_status'].isna().sum()

### giving every user at least "bronze" status - because the voting participants did not necessarily participated in the forum discussion

In [ ]:
df_merged_vote['NFT_status'] = df_merged_vote["NFT_status"].fillna('Bronze')

In [ ]:
df_merged_vote['NFT_status'].value_counts()

In [ ]:
#changing NAN to 0
df_merged_vote['score'].fillna(0)

### filling up the missing addresses (mutually) - if the participant didn't have wallet address, but had NTF address, it's inserted and vica versa

In [ ]:
# 0 changed to NAN
df_merged_vote['add'] = df_merged_vote['add'].replace({0:np.nan})

In [ ]:
# NAN replaced with the other address
df_merged_vote['add'] = df_merged_vote['add'].fillna(df_merged_vote['nft_add'])

In [ ]:
# 0 changed to NAN
df_merged_vote['nft_add'] = df_merged_vote['nft_add'].replace({0:np.nan})

In [ ]:
# NAN replaced with the other address
df_merged_vote['nft_add'] = df_merged_vote['nft_add'].fillna(df_merged_vote['add'])

In [ ]:
#sorting values
df_merged_vote = (df_merged_vote.sort_values(by='score', ascending=False)).reset_index(drop=True)

### check duplicates and drop them, if it's necessary

df_merged_vote['add'].value_counts()

df_merged_vote.drop_duplicates(subset=['add'])

### creating the list for HOPR distribution

In [ ]:
#creating HOPR token format from the score
df_merged_vote['score_h'] = df_merged_vote['score']*1e18

In [ ]:
#subfile for HOPR allocation
hopr_allocation = df_merged_vote.filter(['add', "score", "score_h",], axis=1)

In [ ]:
#dropping NANs
hopr_allocation = hopr_allocation.dropna()

In [ ]:
#changint the datatype of the score to int
hopr_allocation["score_h"] = hopr_allocation['score_h'].apply(lambda x: int(x))
hopr_allocation

In [ ]:
#creating a new variable from the score as str 
hopr_allocation["score_sth"] = hopr_allocation['score_h'].apply(lambda x: str(x))

In [ ]:
#creating an array from the ammount
hopr_dist_score_h = hd['score_sth'].to_numpy()

In [ ]:
#creating an array from the address
hopr_dist_add = hd['add'].to_numpy()

### NFT distribution

In [ ]:
#subframe
nft_distribution = df_merged_vote.filter(['nft_add', "NFT_status"], axis=1)

In [ ]:
#creating a column with the right format
nft_distribution['eoa'] = '>' + nft_distribution["nft_add"] + '<'

In [ ]:
#adding a column as for the status
nft_distribution['grade'] = nft_distribution['NFT_status']

In [ ]:
#keeping only the necessary columns
nft_distribution = nft_distribution.drop(['nft_add', 'NFT_status'], axis=1)

In [ ]:
nft_distribution = nft_distribution.dropna()
nft_distribution

In [ ]:
#saving
nft_distribution.to_csv('daov2_nft_distribution.csv')